In [62]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import pickle

In [63]:
car=pd.read_csv('Cleaned_Car_data.csv')

In [64]:
car.head()

In [65]:
X=car.drop(columns='Price')
y = car['Price']

In [66]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [67]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

In [68]:
ohe.categories_

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [70]:
# Fit the OneHotEncoder
ohe = OneHotEncoder()
ohe.fit(X[['name', 'company', 'fuel_type']])

In [71]:
column_trans = make_column_transformer(
    (OneHotEncoder(categories=ohe.categories_), ['name', 'company', 'fuel_type']),
    remainder='passthrough'
)

In [72]:
# Fit the model using pipeline
lr = LinearRegression()
pipe = make_pipeline(column_trans, lr)
pipe.fit(X_train, y_train)

In [73]:
y_pred = pipe.predict(X_test)

In [74]:
# Calculate R-squared score
r2 = r2_score(y_test, y_pred)

In [75]:
# Save the model using pickle
with open('LinearRegressionModel.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [84]:
# Generate a sample input for prediction
sample_input = np.array(['Maruti Suzuki Swift', 'Maruti', 2019, 1000,'Petrol']).reshape(1, -1)
sample_df = pd.DataFrame(columns=['name', 'company', 'year', 'kms_driven', 'fuel_type'], data=sample_input)

# Load the trained model using pickle
with open('LinearRegressionModel.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Predict the price using the loaded model
predicted_price = loaded_model.predict(sample_df)

# Extract the factors used in prediction
factors_used = sample_df.columns.tolist()

# Create a description of the output including information about factors affecting price
description = f"The predicted price for a {sample_df['name'].values[0]} ({sample_df['year'].values[0]}) with {sample_df['kms_driven'].values[0]} kms driven and {sample_df['fuel_type'].values[0]} fuel type is {predicted_price[0]}. Factors affecting the price are:"

# Iterate through each factor and print its impact on the price
for factor in factors_used:
    if factor in ['name', 'company', 'fuel_type']:
        continue  # Skip categorical factors during this analysis
    factor_df = sample_df.copy()
    factor_df[factor] = factor_df[factor].mean()  # Use the mean value for numerical factors
    
    # Predict the price with the transformed data
    factor_price = loaded_model.predict(factor_df)
    price_difference = predicted_price[0] - factor_price[0]
    print(f"- {factor}: Reduces price by {abs(price_difference)}")

# Print the description
print(description)
